#### Building Pulse Instructions

Pulse programs, which are called `Schedule`s, describe **instruction sequences** for the control electronics.

`Instruction`s available through Qiskit Pulse:
  - [`Delay(duration: int, channel)`](#delay)
  - [`Play(pulse, channel)`](#play)
  - [`SetFrequency(frequency, channel)`](#frequency)
  - [`ShiftPhase(phase, channel)`](#phase)
  - [`Acquire(duration, channel, mem_slot, reg_slot)`](#acquire)
  
Each instruction type has its own set of operands. As you can see above, they each include at least one `Channel` to specify where the instruction will be applied.

**Channels** are labels for signal lines from the control hardware to the quantum chip.

 - `DriveChannel`s are typically used for _driving_ single qubit rotations,
 - `ControlChannel`s are typically used for multi-qubit gates or additional drive lines for tunable qubits, 
 - `MeasureChannel`s are specific to transmitting pulses which stimulate readout, and
 - `AcquireChannel`s are used to trigger digitizers which collect readout signals.
 
`DriveChannel`s, `ControlChannel`s, and `MeasureChannel`s are all `PulseChannel`s; this means that they support _transmitting_ pulses, whereas the `AcquireChannel` is a receive channel only and cannot play waveforms.

For the following examples, we will create one `DriveChannel` instance for each `Instruction` that accepts a `PulseChannel`. Channels take one integer `index` argument. Except for `ControlChannel`s, the index maps trivially to the qubit label.

In [5]:
from qiskit.pulse import DriveChannel

channel = DriveChannel(0) # lables the signal line driving the qubit labeled with index 0

#### Delay

One of the simplest instructions we can build is `Delay`. This is a blocking instruction that tells the control electronics to output no signal on the given channel for the duration specified. It is useful for controlling the timing of other instructions.

The duration here and elsewhere is in terms of the backend's cycle time (1 / sample rate), `dt`.

To build a `Delay` instruction, we pass the duration and channel:

In [7]:
from qiskit.pulse import Delay

delay_5dt = Delay(5, channel)

where `channel` can be any kind of channel, including `AcquireChannel`.

That's all there is to it. This instruction, `delay_5dt`, is ready to be included in a `Schedule`. Any instruction appended after `delay_5dt` on the same channel will execute five timesteps later than it would have without this delay.


#### Play

The `Play` instruction is responsible for executing _pulses_. It's straightforward to build one:

```
play = Play(pulse, channel)
```

Let's clarify what the `pulse` argument is and explore a few different ways to build one.

#### Pulses

A `Pulse` specifies an arbitrary pulse _envelope_. The modulation frequency and phase of the output waveform are controlled by the `SetFrequency` and `ShiftPhase` instructions, which we will cover next.

The image below may provide some intuition for why they are specified separately. Think of the pulses which describe their envelopes as input to an arbitrary waveform generator (AWG), a common lab instrument -- this is depicted in the left image. Notice the limited sample rate discritizes the signal. The signal produced by the AWG may be mixed with a continuous sine wave generator. The frequency of its output is controlled by instructions to the sine wave generator; see the middle image. Finally, the signal sent to the qubit is demonstrated by the right side of the image below.

**Note**: The hardware may be implemented in other ways, but if we keep the instructions separate, we avoid losing explicit information, such as the value of the modulation frequency.

![alt text](pulse_modulation.png "Pulse modulation image")

There are many methods available to us for building up pulses. Our `library` within Qiskit Pulse contains helpful methods for building `Pulse`s. Let's take for example a simple Gaussian pulse -- a pulse with its envelope described by a sampled Gaussian function. We arbitrarily choose an amplitude of 1, standard deviation $\sigma$ of 10, and 128 sample points.

**Note**: The maximum amplitude allowed is `1.0`. Most systems also have additional constraints on the minimum and maximum number of samples allowed in a pulse.

0.19.2
